In [1]:
# import gzip
#
# with gzip.open('movie_data.csv.gz', 'r') as gz:
#     text = gz.readlines()
# with open('movie_data.csv', 'w') as f:
#     f.writelines([l.decode('utf-8') for l in text])

In [2]:
import pandas as pd

df = pd.read_csv('movie_data.csv')
df.columns = ['review', 'sentiment']

In [3]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [4]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"Election is a Chinese mob movie, or triads in ...",1
1,I was just watching a Forensic Files marathon ...,0
2,Police Story is a stunning series of set piece...,1


In [5]:
df.shape

(50000, 2)

In [6]:
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet, '
                 'and one and one is two'])
bag = count.fit_transform(docs)
print(count.vocabulary_)
print(bag.toarray())

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}
[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


In [7]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


In [8]:
df.loc[0, 'review'][-50:]

'nd three more acting performances (including Yam).'

In [9]:
from commons import preprocessor

preprocessor(df.loc[0, 'review'][-50:])

'nd three more acting performances including yam '

In [10]:
preprocessor("</a>This :) is : ( а test :-) !")

'this is а test :) :)'

In [11]:
df['review'] = df['review'].apply(preprocessor)

In [12]:
from commons import tokenizer

tokenizer('runners like running and thus they run')
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()


def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]


tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [13]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maxim.korolev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:]
 if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [15]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf': [False],
               'vect__norm': [None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]}]
lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf',
                      LogisticRegression(random_state=1,
                                         solver='liblinear'))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring='accuracy', cv=5, verbose=2, n_jobs=-1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(random_state=1,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've...
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
 

In [17]:
print('Наилучший набор параметров: %s ' % gs_lr_tfidf.best_params_)

Наилучший набор параметров: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x00000221660965E0>} 


In [18]:
print('Правильность при перекрестной проверке: %.3f' % gs_lr_tfidf.best_score_)

Правильность при перекрестной проверке: 0.899


In [19]:
clf = gs_lr_tfidf.best_estimator_
print('Правильность при испытании: %.3f' % clf.score(X_test, y_test))

Правильность при испытании: 0.896


In [20]:
import numpy as np
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')


def tokenizer2(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|Р)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace(' - ', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized


def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)  # пропустить заголовок
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [21]:
next(stream_docs(path='movie_data.csv'))

('"Election is a Chinese mob movie, or triads in this case. Every two years an election is held to decide on a new leader, and at first it seems a toss up between Big D (Tony Leung Ka Fai, or as I know him, ""The Other Tony Leung"") and Lok (Simon Yam, who was Judge in Full Contact!). Though once Lok wins, Big D refuses to accept the choice and goes to whatever lengths he can to secure recognition as the new leader. Unlike any other Asian film I watch featuring gangsters, this one is not an action movie. It has its bloody moments, when necessary, as in Goodfellas, but it\'s basically just a really effective drama. There are a lot of characters, which is really hard to keep track of, but I think that plays into the craziness of it all a bit. A 100-year-old baton, which is the symbol of power I mentioned before, changes hands several times before things settle down. And though it may appear that the film ends at the 65 or 70-minute mark, there are still a couple big surprises waiting. Si

In [22]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [23]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', n_features=2 ** 21, preprocessor=None, tokenizer=tokenizer2)
clf = SGDClassifier(loss='log', random_state=1)
doc_stream = stream_docs(path='movie_data.csv')

import pyprind

pbar = pyprind.ProgBar(45)
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:22


In [24]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Правильность: %.3f' % clf.score(X_test, y_test))

Правильность: 0.866


In [25]:
clf = clf.partial_fit(X_test, y_test)

In [26]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english', max_df=.1, max_features=5000)
X = count.fit_transform(df['review'].values)

In [28]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10, random_state=123, learning_method='batch')
X_topics = lda.fit_transform(X)

In [29]:
lda.components_.shape

(10, 5000)

In [30]:
n_top_words = 5
feature_names = count.get_feature_names()
for topic_idx, topic in enumerate(lda.components_):
    print("Teмa %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i] for i in topic.argsort()
    [:-n_top_words - 1:-1]]))

Teмa 1:
worst minutes awful script stupid
Teмa 2:
family mother father children girl
Teмa 3:
american war dvd music history
Teмa 4:
human audience cinema art feel
Teмa 5:
police guy car dead murder
Teмa 6:
horror house sex blood gore
Teмa 7:
role performance comedy actor performances
Teмa 8:
series episode episodes tv season
Teмa 9:
book version original effects read
Teмa 10:
action fight guy fun guys


c:\users\maxim.korolev\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [31]:
horror = X_topics[:, 5].argsort()[::-1]
for iter_idx, movie_idx in enumerate(horror[:3]):
    print(' \nФильм ужасов #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], ' ... ')

 
Фильм ужасов #1:
Emilio Miraglia's first Giallo feature, The Night Evelyn Came Out of the Grave, was a great combination of Giallo and Gothic horror - and this second film is even better! We've got more of the Giallo side of the equation this time around, although Miraglia doesn't lose the Gothic horror stylings tha  ... 
 
Фильм ужасов #2:
This film marked the end of the "serious" Universal Monsters era (Abbott and Costello meet up with the monsters later in "Abbott and Costello Meet Frankentstein"). It was a somewhat desparate, yet fun attempt to revive the classic monsters of the Wolf Man, Frankenstein's monster, and Dracula one "la  ... 
 
Фильм ужасов #3:
This film marked the end of the "serious" Universal Monsters era (Abbott and Costello meet up with the monsters later in "Abbott and Costello Meet Frankentstein"). It was a somewhat desparate, yet fun attempt to revive the classic monsters of the Wolf Man, Frankenstein's monster, and Dracula one "la  ... 


In [32]:
import pickle
import os

dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)
pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)